# Assignment 2

### Nick Sciarrilli
#### Comp 4448
#### 09.13.2021

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Question 1

You are provided the following dataset:

|refund | marital_status | income_above_80k | cheat |
|-------:|--------------:|-----------------:|------:|
| yes | single | yes | no |
| no | married | yes | no |
| no | single | no | no |
| yes | married |yes | no |
| no | divorced | yes | yes |
| no | married | no | no |
| yes | divorced | yes | no |
| no | single | yes | yes |
| no | married | no | no |
| no | single | yes | yes |

The data is to be used to predict whether individuals will cheat in filing their taxes or not. The attributes are refund, indicating whether an individual received tax refund or not, marital_status indicating whether the individual is married, single, or divorced, the income_above_80k, indicating whether an individual’s taxable income is above $80,000 or not. The output variable, cheat, is a binary variable indicating whether an individual cheated in filing taxes or not. 

The following decision tree can be used to extract the classification rule for the data provided. 

![decision tree](images/tree.png)

In [2]:
taxes = pd.read_csv("a2_data.csv")
df = taxes.copy()
taxes

,refund,marital_status,income_above_80k,cheat
0,yes,single,yes,no
1,no,married,yes,no
2,no,single,no,no
3,yes,married,yes,no
4,no,divorced,yes,yes
5,no,married,no,no
6,yes,divorced,yes,no
7,no,single,yes,yes
8,no,married,no,no
9,no,single,yes,yes


**a)** Write an IF-THEN rule derived from thiscopyision tree, that can be used to classify instances in the data suppose your desire is to predict “yes” for the output variable.

In [3]:
def rule_tree(row):
    if row.refund == "yes":
        return "no"
    elif row.income_above_80k == "no":
        return "no"
    elif row.marital_status == "married":
        return "no"
    else:
        return "yes"


rule_tree(df.iloc[4])

'yes'

**b)** In code, create a function with an if-else statement to implement the rule you wrote above in question 1a. You can name the function predict. The function takes the input data and returns a vector of predicted output values. You can decide if you want your function’s input data argument to be a numpy array or a data frame, ,then structure your function body to process the input data accordingly. Provide some brief document of your function. You can loop through each instance in the input data and assess whether the instance satisfies the rule for predicting “yes” or not. If the instance satisfies the rule for predicting “yes”, then predict 1, otherwise, predict 0. 

In [4]:
def rule_tree(row):
    if row.refund == "yes":
        return 0
    elif row.income_above_80k == "no":
        return 0
    elif row.marital_status == "married":
        return 0
    else:
        return 1


def predict(data):
    """
    The predict function checks if the data is a dataframe or an individual sample. If the input
    data is a dataframe it processes it and returns a numpy array. If the input is an individual sample
    it returns a single value of 0 or 1 predicting if someone will cheat on their taxes.
    """
    if isinstance(data, pd.DataFrame):
        return data.apply(rule_tree, axis=1).values.astype("uint8")
    else:
        return rule_tree(data)


predict(df)

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1], dtype=uint8)

**c)** Create another function that implements the rule for predicting “yes” for the output, but this time, you will not use if-else statement or a loop. Rather use numpy arrays, matrices or vector to vectorize your code for faster and more efficient implementation. If the instance satisfies the rule for predicting “yes”, then predict 1, otherwise, predict 0. You can use logical operators for such implementation to compare values of test instances to values of attributes in the rule. This function still takes the input data and returns a vector of predicted values. You can call this function vectorized_predict. 

In [5]:
taxes.iloc[:, 0:3]

,refund,marital_status,income_above_80k
0,yes,single,yes
1,no,married,yes
2,no,single,no
3,yes,married,yes
4,no,divorced,yes
5,no,married,no
6,yes,divorced,yes
7,no,single,yes
8,no,married,no
9,no,single,yes


In [6]:
def vectorized_predict(data):
    # setup cases for positive prediction
    case1 = np.array([0, 0, 0, 1], dtype=np.uint8)
    case2 = np.array([0, 0, 1, 1], dtype=np.uint8)

    # transform dataframe to numerical
    data = data.iloc[:, 0:3]
    dummies = pd.get_dummies(data, drop_first=True)

    # make predictions
    preds = []
    for idx, row in dummies.iterrows():
        if np.array_equal(row.values, case1) | np.array_equal(row.values, case2):
            # print(1)
            preds.append(1)
        else:
            # print(0)
            preds.append(0)
    return np.array(preds, dtype=np.uint8)


vectorized_predict(df)

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1], dtype=uint8)

**d)** Select the input data (refund, marital_status, income_above_80k) in the training data and apply your predict function to the input data to predict the outcome values of the input data. Your function should return a vector of predicted values. 

In [7]:
X = df[["refund", "marital_status", "income_above_80k"]]
predict(X)

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1], dtype=uint8)

**e)** Select the input data (refund, marital_status, income_above_80k) in the training data and apply your vectorized_predict function to the input data to predict the outcome values of the input data. Your function should return a vector of predicted values. Are the predicted values with the vectorized_predict function the same as the predicted values obtained through the predict function? (you should have the same results). 

In [8]:
vectorized_predict(X)

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1], dtype=uint8)

**f)** Include the predicted values as a column to the training data and name that column predicted_cheat. 

In [9]:
X["predicted_cheat"] = vectorized_predict(X)
X

,refund,marital_status,income_above_80k,predicted_cheat
0,yes,single,yes,0
1,no,married,yes,0
2,no,single,no,0
3,yes,married,yes,0
4,no,divorced,yes,1
5,no,married,no,0
6,yes,divorced,yes,0
7,no,single,yes,1
8,no,married,no,0
9,no,single,yes,1


**g)** Create a function that computes the overall accuracy of the classification. You can call this function overall_accuracy. The function should take two arguments, a vector of  actual output values and a vector of predicted output values, then returns the classification accuracy. Inside the function, you can use boolean logic to compare the actual and predicted values, then count the proportion of how many predicted values are equal to the actual values to get the overall accuracy. 

In [10]:
labels = taxes["cheat"]
y_true = pd.get_dummies(labels, drop_first=True).squeeze().values
# y_pred = vectorized_predict(X)
y_pred = np.array([1, 0, 0, 0, 1, 0, 0, 1, 0, 1]).astype("uint8")


def overall_accuracy(y_true, y_pred):
    compare = y_true == y_pred
    count = 0
    for i in compare:
        if i == True:
            count += 1
    return count / len(compare)


overall_accuracy(y_true, y_pred)

0.9

**h)** Create a confusion matrix using the predicted and actual output values. You can use pandas crosstab function. 

In [11]:
pd.crosstab(y_true, y_pred, rownames=["actual"], colnames=["predicted"])

predicted,0,1
actual,,
0,6,1
1,0,3


**i)** Supose the test set data is as follows. 

|refund | marital_status | income_above_80k | cheat |
|-------:|--------------:|-----------------:|------:|
| no | single | yes | no |
| no | single | yes | no |
| no | married | yes | no |
| no | divorced |no | no |
| no | married | yes | no |
| no | single | yes | no |
| yes | single | yes | no |
| no | single | yes | yes |
| yes | married | yes | yes |
| yes | single | no | yes |

Apply the vectorized_predict function to the input of the test dataset to predict the output values for the test dataset. Include these predicted values as a column to the test dataset and called this column predicted_cheat. 


In [12]:
test = pd.read_csv("a2_data_i.csv")
y_pred = vectorized_predict(test)
test["predicted_cheat"] = y_pred
test

,refund,marital_status,income_above_80k,cheat,predicted_cheat
0,no,single,yes,no,1
1,no,single,yes,no,1
2,no,married,yes,no,0
3,no,divorced,no,no,0
4,no,married,yes,no,0
5,no,single,yes,no,1
6,yes,single,yes,no,0
7,no,single,yes,yes,1
8,yes,married,yes,yes,0
9,yes,single,no,yes,0


**j)** Apply the overall_accuracy function to the predicted out values and actual output values of the test set to compute the overall accuracy. Compare the overall accuracy of the test set and that of the training set. Is there overfitting? Why or why not? If there is overfitting, what would you do to avoid overfitting the rule to the training set? 

In [13]:
labels = test["cheat"]
y_true = pd.get_dummies(labels, drop_first=True).squeeze().values
overall_accuracy(y_true, y_pred)

0.5

#### Response
Some of the variables from the test set are not conforming to the original decision tree logic. For example, sample on index 0 should be predicted as cheat according to the orinal decision tree of refund=no, marital_status=single, and income_above_80k=yes. Also sample on index 9, should be predicted as cheat of no because the marital_status=married. 

## Question 2 
You will use the same training dataset provided in question 1. Suppose we wanted to create one rule using only one attribute that best classifies the input data, you will need to write a function or an algorithm to find that best attribute. There are different approaches we can use to evaluate the best attritubute that will classify our data with the highest accuracy. One of the accuracy measures we can use to find the best attribute is information gain, which uses entropy. 

**a)** Given the training dataset in question 1, write a function that computes the entropy of output variable. The function should take the entire training dataset, and the name of the output variable as arguments and return the entropy value. You can call the function, entropy. 
Note that entropy H(S) of the target variable T with possible values $ C_i $ for a set of S examples is given by:

$ H(S) =  \sum_{i=1}^{c} p_{i}\log_{2}p_{i} $


where $ p_i $ is proportion of class $ c_i $  in the S examples 

Alternatively, you could also define entropy for a binary target variable as:

$ Entropy = -p \log_{2}p-(1-p)\log_{2}(1-p) $
 
Where p = proportion of the desired class or proportion of “yes” in the given data. 

Apply the entropy function to the training dataset to compute the entropy. Do you think the data is more pure, less pure or more/less pure?


In [14]:
labels = taxes["cheat"]
y_true = pd.get_dummies(labels, drop_first=True).squeeze().values

In [15]:
p = y_true.sum() / len(y_true)

In [16]:
from math import log2


def entropy(data, target):
    # convert label column to numbers
    labels = data[target]
    labels = pd.get_dummies(labels, drop_first=True).squeeze().values

    value, counts = np.unique(labels, return_counts=True)
    probs = counts / len(labels)

    ent = 0
    # calc entropy
    for prob in probs:
        if prob > 0:
            # use log from math and set base to 2
            ent += prob * log2(prob)
    return -ent


entropy(taxes, "cheat")

0.8812908992306927

In [17]:
def entropy2(data, target):
    # convert label column to numbers
    labels = data[target]
    labels = pd.get_dummies(labels, drop_first=True).squeeze().values

    # calculate p
    p = labels.sum() / len(labels)

    # calc entropy
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)


entropy2(taxes, "cheat")

0.8812908992306927

**b)** Create another function that takes the dataset, the name of the output variable, and a specific input variable as arguments and returns the information gain for a split of the data on that specific input variable. You can call the function, information_gain. Information gain is defined as:

Entropy before split - weighted entropy after split

$ Gain(S,A) = H(s) - \sum_{v\in Value(A)} \frac{|s_v|}{|s|}H(s_v) $

A: candidate attribute <br>
v: possible values of A <br>
S: set of examples <br>
$ S_v $: subset of examples where A=v <br>

 
Apply the information_gain function to each input variable to compute the information gain for each input variable. Which input variables is the best (has the highest information gain) for creating one-rule? 


In [18]:
dummies = pd.get_dummies(taxes, drop_first=True)
dummies.columns = ["refund", "married", "single", "80k", "cheat"]

In [19]:
def information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = entropy(data, target_name)

    # Find the unique values in the column
    values = data[split_name].unique()

    # Make two subsets of the data, based on the unique values
    left_split = data[data[split_name] == values[0]]

    # print('left')
    # print(left_split)

    right_split = data[data[split_name] == values[1]]
    # print('right')
    # print(right_split)

    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = subset.shape[0] / data.shape[0]
        # print('subset')
        # print(subset)
        # print(entropy(subset, target_name))
        to_subtract += prob * entropy(subset, target_name)

    # Return information gain
    return original_entropy - to_subtract

In [20]:
information_gain(dummies, "refund", "cheat")

0.19163120400671663

In [21]:
information_gain(dummies, "80k", "cheat")

0.19163120400671663

In [22]:
information_gain(dummies, "married", "cheat")

0.2812908992306927

In [23]:
information_gain(dummies, "single", "cheat")

0.09127744624168022

**c)** Do a cross tabulation using the best attribute obtained in 2b above and the output variable in the training dataset. 

In [24]:
labels = taxes["cheat"]
y_true = pd.get_dummies(labels, drop_first=True).squeeze().values
y_pred = np.where(taxes.marital_status == "married", 1, 0)
pd.crosstab(y_true, y_pred, rownames=["actual"], colnames=["predicted"])

predicted,0,1
actual,,
0,3,4
1,3,0


**d)** Draw a decision tree (use shapes in the word document) to show how the best variable can be used for classification. This is a decision tree with a single node or single variable. 

![married tree](images/married.png)

**e)** Create a one-rule from the decision tree in 2d. The rule should containing an antecedent and a consequent. The antecedent should use only the best attribute and it’s value or values. 

In [25]:
for rec in taxes.marital_status:
    # antecedent
    if rec == "married":
        # consequent
        print(rec, "not cheat")
    else:
        # consequent
        print(rec, "cheat")

single cheat
married not cheat
single cheat
married not cheat
divorced cheat
married not cheat
divorced cheat
single cheat
married not cheat
single cheat


**f)**	Using the antecedent of your rule, extract the data covered by the rule and compute the coverage of the rule. 

In [26]:
taxes.marital_status

0      single
1     married
2      single
3     married
4    divorced
5     married
6    divorced
7      single
8     married
9      single
Name: marital_status, dtype: object

In [27]:
for rec in taxes.marital_status:
    # antecedent
    if rec == "married":
        # consequent
        print("not cheat")
    else:
        # consequent
        print("cheat")

cheat
not cheat
cheat
not cheat
cheat
not cheat
cheat
cheat
not cheat
cheat


In [28]:
y_pred = []
for rec in taxes.marital_status:
    # antecedent
    if rec == "married":
        # consequent
        y_pred.append(0)
    else:
        # consequent
        y_pred.append(1)


y_pred = np.array(y_pred)
coverage = sum(np.where(y_true == y_pred, 1, 0)) / len(y_pred)
print(f"coverage: {coverage}")

coverage: 0.7


**g)**	Using the antecedent of your rule, extract the data covered by the rule and compute the accuracy of the rule. 

In [29]:
y_pred = []
for rec in taxes.marital_status:
    # antecedent
    if rec == "married":
        # consequent
        y_pred.append(0)
    else:
        # consequent
        y_pred.append(1)


y_pred = np.array(y_pred)
overall_accuracy(y_true, y_pred)
# print(y_true == y_pred)

0.7

**h)**	Create a function that implents the rule as an if-else statement to predict the outcomes of any instance (a vector of values associated with the input variables for a specific individual). The function should be able to take one or more instances as an argument in the form of a dataframe or numpy arrary. 

In [30]:
if isinstance(taxes, pd.DataFrame):
    taxes.apply(rule_tree, axis=1).values.astype("uint8")

In [31]:
taxes.shape

(10, 4)

In [32]:
taxes.loc[0].shape

(4,)

In [33]:
def rule(data):
    # if data is a single instance
    if data.shape == (4,):
        # antecedent
        if data[1] == "married":
            # consequent
            y_pred = [0]
        else:
            # consequent
            y_pred = [1]
        y_pred = np.array(y_pred)
    # if data is a dataframe of multiple instances
    else:
        y_pred = []
        for rec in data.marital_status:
            # antecedent
            if rec == "married":
                # consequent
                y_pred.append(0)
            else:
                # consequent
                y_pred.append(1)
        y_pred = np.array(y_pred)
    # return predictions
    return y_pred

In [34]:
# test rule on entire dataframe
rule(taxes)

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1])

In [35]:
# test rule on a single instance
rule(taxes.loc[0])

array([1])

In [36]:
# test rule on a single instance
rule(taxes.loc[1])

array([0])

In [37]:
# test rule on a single instance
rule(taxes.loc[2])

array([1])

**i)**	Implemement the function on the training set and test set in question 1 to get the predicted outputs for the training set and test set.  

In [38]:
training_pred = rule(taxes)
training_pred

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1])

In [39]:
test_pred = rule(test)
test_pred

array([1, 1, 0, 1, 0, 1, 1, 1, 0, 1])

**j)**	What is the overall prediction accuracies for the training set and test set? You can use the overall_accuracy function you initially defined. 

In [40]:
# train y_true
labels = taxes["cheat"]
train_y_true = pd.get_dummies(labels, drop_first=True).squeeze().values

# test y_true
labels = test["cheat"]
test_y_true = pd.get_dummies(labels, drop_first=True).squeeze().values

In [41]:
train_y_true

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1], dtype=uint8)

In [42]:
test_y_true

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1], dtype=uint8)

In [43]:
# training data accuracy
overall_accuracy(train_y_true, training_pred)

0.7

In [44]:
# test data accuracy
overall_accuracy(test_y_true, test_pred)

0.4

**k)**	Do you think there is overfitting or underfitting? Why or why not? 

Yes it looks like our single rule classifier is overfitting because it has a higher accuracy on the training data than it does on the test data.